In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

from scipy.stats import percentileofscore
from pathlib import Path

In [2]:
base_dir= Path.cwd().parent
data_dir = base_dir/"data"
cleaned = data_dir/"cleaned"
processed = data_dir/"processed"

In [ ]:
df = pd.read_csv(
    processed/"master_monthly.csv",
     parse_dates=["date"], dayfirst=True)
df = df.sort_values("date").reset_index(drop=True)
df.head()

In [ ]:
print("Rows:", len(df))
print("Start Date:", df["date"].min())
print("End Date:", df["date"].max())
print("Duplicate dates:", df["date"].duplicated().sum())

In [ ]:
df.columns

In [ ]:
df["fx_rate"] = pd.to_numeric(df["fx_rate"], errors="coerce")
print("Missing INR values:", df["fx_rate"].isna().sum())

In [ ]:
df["inr_dep_mom"] = df["fx_rate"].pct_change()

In [ ]:
df["inr_dep_12m"] = df["fx_rate"].pct_change(12)

In [ ]:
df["inr_dep_36m"] = df["fx_rate"].pct_change(36)

In [ ]:
plt.figure()
plt.plot(df["date"], df["fx_rate"])
plt.title("USD-INR (Monthly Level)")
plt.xlabel("Date")
plt.ylabel("INR per USD")
plt.show()

In [ ]:
plt.figure()
plt.plot(df["date"], df["inr_dep_12m"])
plt.axhline(0)
plt.title("USD-INR: 12-Month Depreciation Rate (YoY)")
plt.xlabel("Date")
plt.ylabel("YoY % change")
plt.show()

In [ ]:
s = df["inr_dep_12m"].dropna()
print("Mean YoY depreciation:",round(s.mean(),3))
print("Median:", round(s.median(),3))
print("Std dev:", round(s.std(),3))
print("\nPercentiles:")
for p in [5,10,25,50,75,90,95]:
    print(f"{p}%:", round(np.percentile(s,p),3))

In [ ]:
latest_dep = s.iloc[-1]
print("Latest YoY depreciation:", round(latest_dep, 3))
pct_rank = percentileofscore(s, latest_dep, kind="weak")
print("\nLatest observation percentile:", round(pct_rank, 1))


In [ ]:
p95 = np.percentile(s, 95)
df["stress_flag"] = df["inr_dep_12m"] >= p95

print("95th percentile YoY depreciation:", round(p95, 3))
print("Number of stress months:", df["stress_flag"].sum())

In [ ]:
df.loc[
    df["stress_flag"],
    ["date", "fx_rate", "inr_dep_12m"]
    ].sort_values("inr_dep_12m", ascending=False).head(20)

In [ ]:
df.to_csv(processed/"inr_stress_test.csv", index=False)

In [ ]:
monthly = pd.read_csv(
    processed/"master_monthly.csv",
     parse_dates=["date"], dayfirst=True)
cpi = pd.read_csv(
    processed/"master_yearly.csv",
     parse_dates=["date"], dayfirst=True)

In [ ]:
monthly["year"] = monthly["date"].dt.year
usd_inr_annual = (
    monthly
    .groupby("year", as_index=False)["fx_rate"]
    .mean()
)

In [ ]:
cpi["year"] = cpi["date"].dt.year
cpi["inflation_diff"] = cpi["cpi_india"] - cpi["cpi_us"]
cpi.head()

In [ ]:
annual = (
    usd_inr_annual
    .merge(
        cpi[["year", "cpi_india", "cpi_us", "inflation_diff"]],
        on="year",
        how="inner"
    )
)
annual.head()

In [ ]:
annual["inr_dep_yoy"] = annual["fx_rate"].pct_change()

In [ ]:
annual["cum_inr_dep"] = (1 + annual["inr_dep_yoy"]).cumprod() - 1
annual["cum_inflation_diff"] = (1 + annual["inflation_diff"]/100).cumprod() - 1

In [ ]:
plt.figure()
plt.plot(annual["year"], annual["cum_inr_dep"], label="Cumulative INR Depreciation")
plt.plot(annual["year"], annual["cum_inflation_diff"], label="Cumulative Infation Differential")

plt.title("INR Depreciation vs India-US Inflation Differential")
plt.xlabel("Year")
plt.ylabel("Cumulative Change")
plt.legend()
plt.show()

In [ ]:
annual.loc[
    annual["year"].isin([1991, 2000, 2010, 2020, annual["year"].max()]),
    ["year", "cum_inr_dep", "cum_inflation_diff"]
    ]


In [ ]:
annual["excess_dep"] = annual["inr_dep_yoy"] - (annual["inflation_diff"] / 100)

In [ ]:
plt.figure()
plt.bar(annual["year"], annual["excess_dep"])
plt.axhline(0)
plt.title("Excess INR Depreciation vs Inflation Differential (Annual)")
plt.xlabel("Year")
plt.ylabel("Excess depreciation")
plt.show()

In [ ]:
annual["excess_dep_5y_avg"] = annual["excess_dep"].rolling(5).mean()

plt.figure()
plt.plot(annual["year"], annual["excess_dep_5y_avg"])
plt.axhline(0)
plt.title("5-Year Average Excess INR Depreciation")
plt.xlabel("Year")
plt.ylabel("Average excess depreciation")
plt.show()


In [ ]:
monthly = pd.read_csv(
    processed/"master_monthly.csv",
     parse_dates=["date"], dayfirst=True
).sort_values("date").reset_index(drop=True)

In [ ]:
for col in ["fx_rate", "dxy", "us10y_real"]:
    monthly[col] = pd.to_numeric(monthly[col], errors="coerce")

In [ ]:
monthly["inr_dep_12m"] = monthly["fx_rate"].pct_change(12)

In [ ]:
base = monthly.dropna(subset=["inr_dep_12m", "dxy"]).iloc[0]

monthly["dxy_indexed"] = monthly["dxy"] / base["dxy"] * 100
monthly["usd_inr_indexed"] = monthly["fx_rate"] / base["fx_rate"] * 100

In [ ]:
monthly["usd_inr_12m"] = monthly["fx_rate"].pct_change(12)
monthly["dxy_12m"] = monthly["dxy"].pct_change(12)

plt.figure()
plt.plot(monthly["date"], monthly["usd_inr_12m"], label="INR YoY depreciation")
plt.plot(monthly["date"], monthly["dxy_12m"], label="DXY YoY change")
plt.axhline(0)
plt.legend()
plt.title("YoY INR Depreciation vs YoY Dollar Strength")
plt.show()

In [ ]:
window = monthly[monthly["date"] >= "2005-01-01"].copy()

window["usd_inr_idx"] = window["fx_rate"] / window["fx_rate"].iloc[0] * 100
window["dxy_idx"] = window["dxy"] / window["dxy"].iloc[0] * 100

plt.figure()
plt.plot(window["date"], window["usd_inr_idx"], label="USD–INR")
plt.plot(window["date"], window["dxy_idx"], label="DXY")
plt.legend()
plt.title("USD–INR vs DXY (Indexed from 2005)")
plt.show()


In [ ]:
plt.figure()
plt.plot(monthly["date"], monthly["usd_inr_indexed"], label="USD–INR (indexed)")
plt.plot(monthly["date"], monthly["dxy_indexed"], label="DXY (indexed)")
plt.title("USD–INR and Global Dollar Strength (Indexed)")
plt.xlabel("Date")
plt.ylabel("Index (base = 100)")
plt.legend()
plt.show()


In [ ]:
monthly["dxy_12m"] = monthly["dxy"].pct_change(12)

plt.figure()
plt.scatter(
    monthly["dxy_12m"],
    monthly["inr_dep_12m"],
    alpha=0.5
)
plt.axhline(0)
plt.axvline(0)
plt.title("YoY INR Depreciation vs YoY DXY Change")
plt.xlabel("DXY YoY change")
plt.ylabel("INR YoY depreciation")
plt.show()


In [ ]:
quads = [
    (monthly["dxy_12m"] > 0) & (monthly["inr_dep_12m"] > 0), # Q1: Top-Right
    (monthly["dxy_12m"] < 0) & (monthly["inr_dep_12m"] > 0), # Q2: Top-Left
    (monthly["dxy_12m"] < 0) & (monthly["inr_dep_12m"] < 0), # Q3: Bottom-Left
    (monthly["dxy_12m"] > 0) & (monthly["inr_dep_12m"] < 0)  # Q4: Bottom-Right
]

quad_labels = ["Q1 (Upper-Right)", "Q2 (Upper-Left)", "Q3 (Bottom-Left)", "Q4 (Bottom-Right)"]
monthly["quadrant"] = np.select(quads, quad_labels, default="On Axis")

pop_counts = monthly["quadrant"].value_counts().drop("On Axis", errors='ignore')
most_populated = pop_counts.idxmax()
total_points = pop_counts.max()

print(f"The most populated quadrant is: {most_populated} with {total_points} points.")
print("\nFull Distribution:")
print(pop_counts)


In [ ]:
subset = monthly[["dxy_12m", "inr_dep_12m"]].dropna()
corr_coef = subset["dxy_12m"].corr(subset["inr_dep_12m"])
print(f"Correlation between YoY DXY change and YoY INR depreciation: {corr_coef:.2f}")


In [ ]:
ry = monthly.dropna(subset=["us10y_real", "inr_dep_12m"])

In [ ]:
plt.figure()
plt.scatter(
    ry["us10y_real"],
    ry["inr_dep_12m"],
    alpha=0.5
)
plt.axhline(0)
plt.axvline(0)
plt.title("INR Depreciation vs US 10Y Real Yields")
plt.xlabel("US 10Y Real Yield")
plt.ylabel("INR YoY depreciation")
plt.show()


In [ ]:
plt.figure()
plt.plot(monthly["date"], monthly["us10y_real"], label="US 10Y Real Yield")
plt.plot(monthly["date"], monthly["inr_dep_12m"], label="INR YoY depreciation")
plt.axhline(0)
plt.title("INR Depreciation and Global Real Rates")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.show()


In [ ]:
monthly["fx_reserves"] = pd.to_numeric(monthly["fx_reserves"], errors="coerce")


In [ ]:
plt.figure()
plt.plot(monthly["date"], monthly["fx_reserves"])
plt.title("India FX Reserves (Monthly, Ex-Gold)")
plt.xlabel("Date")
plt.ylabel("FX reserves")
plt.show()


In [ ]:
base_date = monthly.dropna(subset=["fx_reserves", "fx_rate"]).iloc[0]

monthly["reserves_idx"] = monthly["fx_reserves"] / base_date["fx_reserves"] * 100
monthly["usd_inr_idx"] = monthly["fx_rate"] / base_date["fx_rate"] * 100

plt.figure()
plt.plot(monthly["date"], monthly["usd_inr_idx"], label="USD–INR (indexed)")
plt.plot(monthly["date"], monthly["reserves_idx"], label="FX reserves (indexed)")
plt.title("USD–INR vs FX Reserves (Indexed)")
plt.xlabel("Date")
plt.ylabel("Index (base=100)")
plt.legend()
plt.show()


In [ ]:
monthly["reserves_12m_change"] = monthly["fx_reserves"].pct_change(
    12,
    fill_method=None    
)

plt.figure()
plt.plot(monthly["date"], monthly["reserves_12m_change"])
plt.axhline(0)
plt.title("FX Reserves: 12-month % change")
plt.xlabel("Date")
plt.ylabel("YoY % change")
plt.show()


In [ ]:
s = monthly["inr_dep_12m"].dropna()
p90 = np.percentile(s, 90)

monthly["inr_stress"] = monthly["inr_dep_12m"] >= p90
print("Stress threshold (95th percentile YoY depreciation):", round(p95, 3))
print("Stress months:", monthly["inr_stress"].sum())

In [ ]:
stress_res = monthly.loc[monthly["inr_stress"], "reserves_12m_change"].dropna()
normal_res = monthly.loc[~monthly["inr_stress"], "reserves_12m_change"].dropna()

print("Median reserves YoY change (stress months):", round(stress_res.median(), 3))
print("Median reserves YoY change (normal months):", round(normal_res.median(), 3))


In [ ]:
plt.figure()
plt.plot(monthly["date"], monthly["fx_reserves"], label="FX reserves")

stress_points = monthly[monthly["inr_stress"] & monthly["fx_reserves"].notna()]
plt.scatter(stress_points["date"], stress_points["fx_reserves"], s=10, label="High INR depreciation months")

plt.title("FX Reserves with INR Stress Months Highlighted")
plt.xlabel("Date")
plt.ylabel("FX reserves")
plt.legend()
plt.show()


In [ ]:
monthly["date"] = pd.to_datetime(monthly["date"], dayfirst=True, errors="coerce")
monthly = monthly.sort_values("date").reset_index(drop=True)

#calculating volatility of the inr against usd

monthly["fx_logret"] = np.log(monthly["fx_rate"]).diff()
monthly["fx_vol_12m"] = monthly["fx_logret"].rolling(12).std() * np.sqrt(12)

In [ ]:
sig = monthly[["date", "inr_dep_12m", "fx_vol_12m", "fx_reserves", "reserves_12m_change","fx_rate"]].copy()

pace_p90 = np.nanpercentile(sig["inr_dep_12m"], 90)
vol_p75  = np.nanpercentile(sig["fx_vol_12m"], 75)
vol_p90  = np.nanpercentile(sig["fx_vol_12m"], 90)
res_p10  = np.nanpercentile(sig["reserves_12m_change"], 10)

print("Depreciation rate @ 90th percentile:", pace_p95)
print("Volitality @ 75th percentile & @ 90th percentile:", vol_p75, vol_p90)
print("Reserves change 10th percentile:", res_p10)


In [ ]:
sig["pace_stress"] = sig["inr_dep_12m"] >= pace_p90
sig["vol_stress"]  = sig["fx_vol_12m"] >= vol_p75
sig["buf_stress"]  = sig["reserves_12m_change"] <= 0 

sig["stress_flag"] = sig["pace_stress"] & (sig["vol_stress"] | sig["buf_stress"])

print(sig[["pace_stress","vol_stress","buf_stress","stress_flag"]].mean().round(3))
print("Stress months:", int(sig["stress_flag"].sum()), "out of", sig["stress_flag"].count())


In [ ]:
sig = sig.sort_values("date").reset_index(drop=True)

sig["stress_start"] = sig["stress_flag"].astype(bool) & ~sig["stress_flag"].astype(bool).shift(1, fill_value=False)
sig["episode_id"] = sig["stress_start"].cumsum()
sig.loc[~sig["stress_flag"], "episode_id"] = np.nan

episodes = (sig.dropna(subset=["episode_id"])
              .groupby("episode_id")
              .agg(start=("date","min"),
                   end=("date","max"),
                   months=("date","count"),
                   max_dep=("inr_dep_12m","max"),
                   max_vol=("fx_vol_12m","max"),
                   min_res_yoy=("reserves_12m_change","min"))
              .sort_values("months", ascending=False))

episodes.head(10)


In [ ]:
latest = sig.dropna(subset=["inr_dep_12m"]).iloc[-1]
last_res = sig.dropna(subset= ["fx_reserves"]).iloc[-1]
print("Latest date:", latest["date"])
print("Latest YoY INR dep:", latest["inr_dep_12m"])
print("Latest FX vol 12m:", latest["fx_vol_12m"])
print("latest reserves date:", last_res["date"])
print("latest reserves level:", last_res["fx_reserves"])
print("Latest reserves YoY:", last_res["reserves_12m_change"])
print("Pace stress:", latest["pace_stress"], "Vol stress:", latest["vol_stress"], "Buffer stress:", latest["buf_stress"])
print("Stress flag:", latest["stress_flag"])


In [ ]:
sig["buf_stress"]  = sig["reserves_12m_change"] <= res_p10 

sig["stress_flag"] = sig["pace_stress"] & (sig["vol_stress"] | sig["buf_stress"])

print(sig[["pace_stress","vol_stress","buf_stress","stress_flag"]].mean().round(3))
print("Stress months:", int(sig["stress_flag"].sum()), "out of", sig["stress_flag"].count())


In [ ]:
sig = sig.sort_values("date").reset_index(drop=True)

sig["stress_start"] = sig["stress_flag"].astype(bool) & ~sig["stress_flag"].astype(bool).shift(1, fill_value=False)
sig["episode_id"] = sig["stress_start"].cumsum()
sig.loc[~sig["stress_flag"], "episode_id"] = np.nan

episodes = (sig.dropna(subset=["episode_id"])
              .groupby("episode_id")
              .agg(start=("date","min"),
                   end=("date","max"),
                   months=("date","count"),
                   max_dep=("inr_dep_12m","max"),
                   max_vol=("fx_vol_12m","max"),
                   min_res_yoy=("reserves_12m_change","min"))
              .sort_values("months", ascending=False))

episodes.head(10)

In [ ]:
latest = sig.dropna(subset=["inr_dep_12m"]).iloc[-1]
last_res = sig.dropna(subset= ["fx_reserves"]).iloc[-1]
print("Latest date:", latest["date"])
print("Latest YoY INR dep:", latest["inr_dep_12m"])
print("Latest FX vol 12m:", latest["fx_vol_12m"])
print("latest reserves date:", last_res["date"])
print("latest reserves level:", last_res["fx_reserves"])
print("Latest reserves YoY:", last_res["reserves_12m_change"])
print("Pace stress:", latest["pace_stress"], "Vol stress:", latest["vol_stress"], "Buffer stress:", latest["buf_stress"])
print("Stress flag:", latest["stress_flag"])


In [ ]:
pace_p75 = np.nanpercentile(sig["inr_dep_12m"], 75)
pace_p90 = np.nanpercentile(sig["inr_dep_12m"], 90)

vol_p75  = np.nanpercentile(sig["fx_vol_12m"], 75)
vol_p90  = np.nanpercentile(sig["fx_vol_12m"], 90)

res_p10  = np.nanpercentile(sig["reserves_12m_change"], 10)

sig["stress_regime"] = 0

# Stress (single gate)
sig.loc[
    (sig["inr_dep_12m"] >= pace_p90) &
    ((sig["fx_vol_12m"] >= vol_p90) | (sig["reserves_12m_change"] <= res_p10)),
    "stress_regime"
] = 2

# Elevated pressure (single gate, only if not stress)
sig.loc[
    (sig["stress_regime"] == 0) &
    (sig["inr_dep_12m"] >= pace_p75) &
    ((sig["fx_vol_12m"] >= vol_p75) | (sig["reserves_12m_change"] <= 0)),
    "stress_regime"
] = 1


In [ ]:
# final plot 1: usd-inr with stress and elevated pressure periods highlighted

sig = sig.copy()
sig["date"] = pd.to_datetime(sig["date"], dayfirst=True, errors="coerce")
sig = sig.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

# regime definitions (single-gate)
pace_p75 = np.nanpercentile(sig["inr_dep_12m"], 75)
pace_p90 = np.nanpercentile(sig["inr_dep_12m"], 90)

vol_p75  = np.nanpercentile(sig["fx_vol_12m"], 75)
vol_p90  = np.nanpercentile(sig["fx_vol_12m"], 90)

res_p10  = np.nanpercentile(sig["reserves_12m_change"], 10)

sig["stress_regime"] = 0

# stress: dep>=90p & (vol>=90p | reserves<=10p)
sig.loc[
    (sig["inr_dep_12m"] >= pace_p90) &
    ((sig["fx_vol_12m"] >= vol_p90) | (sig["reserves_12m_change"] <= res_p10)),
    "stress_regime"
] = 2

# elevated pressure: dep>=75p & (vol>=75p | reserves<=0)
sig.loc[
    (sig["stress_regime"] == 0) &
    (sig["inr_dep_12m"] >= pace_p75) &
    ((sig["fx_vol_12m"] >= vol_p75) | (sig["reserves_12m_change"] <= 0)),
    "stress_regime"
] = 1

fig, ax = plt.subplots(figsize=(12, 6))
#usd inr line
ax.plot(sig["date"], sig["fx_rate"], color="steelblue", linewidth=1.5)

tmp = sig[["date", "stress_regime"]].copy()

#extreme stress block
is_stress = tmp["stress_regime"] == 2
stress_start = is_stress & ~is_stress.shift(1, fill_value=False)
episode_id = stress_start.cumsum()
episode_id = episode_id.where(is_stress, np.nan)

tmp["episode_id"] = episode_id

episodes = (
    tmp.dropna(subset=["episode_id"])
       .groupby("episode_id")
       .agg(start=("date", "min"), end=("date", "max"))
)

for _, r in episodes.iterrows():
    ax.axvspan(
        r["start"],
        r["end"] + pd.offsets.MonthBegin(1),
        color="#d62728",
        alpha=0.25,
        linewidth=0
    )

#elevated pressure block (only if persists for >=2 months)
is_elev = tmp["stress_regime"] == 1
elev_start = is_elev & ~is_elev.shift(1, fill_value=False)
elev_id = elev_start.cumsum()
elev_id = elev_id.where(is_elev, np.nan)

tmp["elev_id"] = elev_id

elev_eps = (
    tmp.dropna(subset=["elev_id"])
       .groupby("elev_id")
       .agg(start=("date", "min"), end=("date", "max"), months=("date", "count"))
)

for _, r in elev_eps.iterrows():
    if r["months"] >= 2:   # persistence filter
        ax.axvspan(
            r["start"],
            r["end"] + pd.offsets.MonthBegin(1),
            color="#ffbf00",
            alpha=0.18,
            linewidth=0
        )

ax.set_title("USD–INR with Currency Stress Episodes")
ax.set_xlabel("Date")
ax.set_ylabel("USD to INR (monthly average in ₹)")

legend_patches = [
    mpatches.Patch(color="#d62728", alpha=0.30, label="Stress Episode"),
    mpatches.Patch(color="#ffbf00", alpha=0.25, label="Elevated Pressure (persistent)"),
]
ax.legend(handles=legend_patches, loc="upper left")

caption = (      
    r"$\mathbf{Elevated\ Pressure:}$ dep≥75th pct with vol≥75th pct or reserve YoY≤0%."
    "\n"
    r"$\mathbf{Stress\ Episode:}$ dep≥90th pct with vol≥90th pct or reserve YoY≤10th pct, persisting for >=2 months."
)

fig.text(0.01, -0.03, caption, ha="left", va="top", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# plot 2: distribution of YoY INR depreciation (stress tail = 90th pct)

s = sig["inr_dep_12m"].dropna()

latest_row = sig.dropna(subset=["inr_dep_12m"]).iloc[-1]
latest = float(latest_row["inr_dep_12m"])
latest_date = pd.to_datetime(latest_row["date"]).strftime("%d-%b-%y")
latest_fx = float(latest_row["fx_rate"]) if "fx_rate" in sig.columns else np.nan

median = float(np.percentile(s, 50))
p90 = float(np.percentile(s, 90))

fig, ax = plt.subplots(figsize=(12, 6))

# histogram + KDE
ax.hist(s, bins=28, density=True, color="steelblue", alpha=0.35)
s.plot(kind="kde", color="darkorange", linewidth=1.6, ax=ax)

xmin = max(float(s.min()), -0.25)
xmax = max(float(s.max()), p90 + 0.25)
ax.set_xlim(xmin, xmax)

ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.grid(axis="x", alpha=0.5)

# stress tail shading (90th pct)
ax.axvspan(p90, ax.get_xlim()[1], color="#d62728", alpha=0.08)

# reference lines
ax.axvline(median, color="black", linestyle="--", linewidth=1.5, zorder=3)
ax.axvline(p90, color="#d62728", linestyle="-.", linewidth=2.0, zorder=3)
ax.axvline(latest, color="black", linewidth=3.0, zorder=4)

ax.set_title("Where Today’s INR Depreciation Sits in History", fontsize=15)

ax.set_xlabel("YoY INR Depreciation")
ax.set_ylabel("Historical frequency")

# annotations
ymax = ax.get_ylim()[1]

ax.annotate(
    f"Median\n{median:.1%} YoY",
    xy=(median, ymax * 0.88),
    xytext=(median - 0.10, ymax * 0.88),
    ha="center", va="top",
    fontsize=11,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="grey", alpha=0.9),
    arrowprops=dict(arrowstyle="-", color="grey", lw=1.0)
)

ax.annotate(
    f"Stress threshold\n{p90:.1%} YoY",
    xy=(p90, ymax * 0.90),
    xytext=(p90 + 0.06, ymax * 0.90),
    ha="left", va="top",
    fontsize=11,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="#d62728", alpha=0.95),
    arrowprops=dict(arrowstyle="-", color="#d62728", lw=1.2)
)

latest_label = f"{latest_date}\n{latest:.1%} YoY"
if not np.isnan(latest_fx):
    latest_label += f"\nUSD/INR ≈ ₹{latest_fx:.1f}"

ax.annotate(
    latest_label,
    xy=(latest, ymax * 0.95),
    xytext=(latest + 0.03, ymax * 0.68),
    ha="left", va="top",
    fontsize=12,
    fontweight="bold",
    bbox=dict(boxstyle="round,pad=0.4", fc="white", ec="black", alpha=0.97),
    arrowprops=dict(arrowstyle="-", color="black", lw=1.2)
)

ax.text(
    p90 + 0.06,
    ymax * 0.22,
    "Stress tail\n(extreme deprecation, rare historical episodes)",
    color="#d62728",
    fontsize=15
)

ax.text(
    p90 + 0.005,
    ymax * 0.01,
    "90th Percentile",
    color="#d62728",
    fontsize=12
)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# final plot 3: stress episodes

## redefining variables
pace_p75 = np.nanpercentile(sig["inr_dep_12m"], 75)
pace_p90 = np.nanpercentile(sig["inr_dep_12m"], 90)

vol_p75  = np.nanpercentile(sig["fx_vol_12m"], 75)
vol_p90  = np.nanpercentile(sig["fx_vol_12m"], 90)

res_p10  = np.nanpercentile(sig["reserves_12m_change"], 10)

sig["stress_regime"] = 0

# stress: single gate
sig.loc[
    (sig["inr_dep_12m"] >= pace_p90) &
    ((sig["fx_vol_12m"] >= vol_p90) | (sig["reserves_12m_change"] <= res_p10)),
    "stress_regime"
] = 2

# elevated pressure (not plotted here)
sig.loc[
    (sig["stress_regime"] == 0) &
    (sig["inr_dep_12m"] >= pace_p75) &
    ((sig["fx_vol_12m"] >= vol_p75) | (sig["reserves_12m_change"] <= 0)),
    "stress_regime"
] = 1

# building from stress regime 2
is_stress = sig["stress_regime"] == 2
stress_start = is_stress & ~is_stress.shift(1, fill_value=False)
sig["episode_id"] = stress_start.cumsum()
sig.loc[~is_stress, "episode_id"] = np.nan

episodes = (
    sig.dropna(subset=["episode_id"])
      .groupby("episode_id")
      .agg(
          start=("date", "min"),
          end=("date", "max"),
          months=("date", "count"),
          peak_dep=("inr_dep_12m", "max"),
          peak_vol=("fx_vol_12m", "max"),
          min_res_yoy=("reserves_12m_change", "min"),
      )
      .sort_values("start")
)

if episodes.empty:
    raise ValueError("No stress episodes found. Check thresholds or inputs.")

# plot 3
fig, ax = plt.subplots(figsize=(10.8, 4.6))

sev = episodes["peak_dep"].to_numpy(dtype=float)
if np.isfinite(sev).sum() >= 2 and np.nanmax(sev) > np.nanmin(sev):
    sev_norm = (sev - np.nanmin(sev)) / (np.nanmax(sev) - np.nanmin(sev))
else:
    sev_norm = np.full_like(sev, 0.5)

alphas = 0.25 + 0.55 * sev_norm

episodes_plot = episodes.copy()
episodes_plot["end_vis"] = episodes_plot["end"] + pd.offsets.MonthBegin(1)

y = np.arange(len(episodes_plot))[::-1]
bar_h = 0.56

xmin = pd.Timestamp("1985-01-01")
xmax = pd.Timestamp("2025-12-01") + pd.DateOffset(months=18)

for i, (_, r) in enumerate(episodes_plot.iterrows()):
    start, end = r["start"], r["end_vis"]
    left = mdates.date2num(start)
    width = mdates.date2num(end) - left

    ax.broken_barh(
        [(left, width)],
        (y[i] - bar_h/2, bar_h),
        facecolors="tab:red",
        edgecolor="none",
        alpha=float(alphas[i]),
        zorder=2
    )

    dep = float(r["peak_dep"])
    dep_pct = dep * 100 if abs(dep) <= 1.5 else dep

    txt1 = f"{int(r['months'])}m | peak dep {dep_pct:.0f}%"
    ax.text(end + pd.Timedelta(days=60), y[i], txt1, va="center", fontsize=9, color="#222222")

yt = []
for r in episodes_plot.itertuples():
    yt.append(f"{r.start.year}" if r.start.year == r.end.year else f"{r.start.year}–{r.end.year}")
ax.set_yticks(y)
ax.set_yticklabels(yt, fontsize=9)

ax.set_xlim(xmin, xmax)
ax.xaxis.set_major_locator(mdates.YearLocator(base=5))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

ax.set_title(
    "INR Stress Episodes: Rare and Clustered",
    loc="center",
    fontsize=13,
    pad=10
)
ax.set_xlabel("Year")

ax.grid(axis="x", alpha=0.12, zorder=1)
ax.tick_params(axis="y", length=0)
ax.tick_params(axis="x", labelsize=9)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["left"].set_color("#cccccc")
ax.spines["left"].set_linewidth(0.8)

caption = (
    "Shading is scaled by severity of currency stress during the period (Darker = Higher Stress)."
)
fig.text(0.01, -0.03, caption, ha="left", va="top", fontsize=10, color="#444444")

plt.tight_layout()
plt.show()

episodes.reset_index(drop=True)


In [ ]:
episodes.head(10)

In [ ]:
vol_p90  = np.nanpercentile(sig["fx_vol_12m"], 90)
count_p90 = np.sum(sig["fx_vol_12m"] >= vol_p90)
print(f"90th percentile (vol): {vol_p90}")
print(f"vol >= 90th percentile: {count_p90}")

In [ ]:
res_p10  = np.nanpercentile(sig["reserves_12m_change"], 10)
count_p10 = np.sum(sig["reserves_12m_change"] <= res_p10)
count_0yoy = np.sum(sig["reserves_12m_change"] <= 0)
print(f"10th percentile (reserves): {res_p10}")
print(f"reserves =< 10th percentile: {count_p10}")
print(f"reserves =< 0 % yoy: {count_0yoy}")

In [ ]:
# final plot 4: inflatiion diff and usd-inr with stress periods highlighted 

sig_plot = sig.copy()
sig_plot["date"] = pd.to_datetime(sig_plot["date"], dayfirst=True, errors="coerce")
sig_plot = sig_plot.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

annual_plot = annual.copy()
annual_plot["year"] = pd.to_numeric(annual_plot["year"], errors="coerce")
annual_plot = annual_plot.dropna(subset=["year"]).sort_values("year").reset_index(drop=True)
annual_plot["year_dt"] = pd.to_datetime(
    annual_plot["year"].astype(int).astype(str) + "-01-01"
)

tmp = sig_plot[["date", "stress_regime"]].copy()

# stress episodes
is_stress = tmp["stress_regime"] == 2
stress_start = is_stress & ~is_stress.shift(1, fill_value=False)
stress_id = stress_start.cumsum()
tmp["stress_id"] = stress_id.where(is_stress, np.nan)

stress_eps = (
    tmp.dropna(subset=["stress_id"])
       .groupby("stress_id")
       .agg(start=("date", "min"), end=("date", "max"))
       .sort_values("start")
)

# plot
fig, ax = plt.subplots(figsize=(10.5, 5))

# Stress shading
for _, r in stress_eps.iterrows():
    ax.axvspan(
        r["start"],
        r["end"] + pd.offsets.MonthBegin(1),
        color="#d62728",
        alpha=0.08,
        linewidth=0,
        zorder=0
    )

# Lines
ax.plot(
    annual_plot["year_dt"],
    annual_plot["cum_inr_dep"],
    linewidth=2.2,
    label="Cumulative INR depreciation vs USD (%)",
    zorder=2
)

ax.plot(
    annual_plot["year_dt"],
    annual_plot["cum_inflation_diff"],
    linewidth=2.2,
    label="Cumulative inflation differential (India – US)",
    zorder=2
)

ax.set_title(
    "Long-run INR Depreciation and Inflation Differential",
    loc="center",
    fontsize=13,
    pad=12
)
ax.set_xlabel("Year")
ax.set_ylabel("Cumulative change since base year (%)")

ax.grid(alpha=0.05)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.xaxis.set_major_locator(mdates.YearLocator(5))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.tick_params(axis="both", labelsize=9)

# Legend
stress_patch = mpatches.Patch(
    color="#d62728", alpha=0.30, label="Stress Episode"
)

line_handles = ax.get_lines()
ax.legend(
    handles=[stress_patch] + line_handles,
    frameon=False,
    loc="upper left",
    fontsize=9
)

# Base year
base_year = int(annual_plot["year"].min())
ax.text(
    0.99, -0.15,
    f"Base year: {base_year}",
    transform=ax.transAxes,
    ha="right",
    fontsize=9,
    color="#555555"
)

caption = (
    "USD–INR is shown as cumulative depriciation from the base year, not the spot exchange rate."
)
fig.text(0.01, -0.03, caption, ha="left", va="top", fontsize=10, color="#444444")

plt.tight_layout()
plt.show()


In [ ]:
p